# 제조 AI 실습 과정 - Session 2 (2-4시간)
## 비전 불량검사 고도화

### 학습 목표
- 데이터 수집/라벨 품질관리 & 조명/렌즈 가이드 이해
- 전이학습+YOLO/세그멘테이션 적용 전략 습득
- 임계값 튜닝과 오탐/미탐 트레이드오프 최적화

---

## 1. 환경 설정 및 라이브러리 설치

한글 깨짐 방지

Step 1: 실행

In [ ]:
!apt-get install -y fonts-nanum*
!rm -rf /root/.cache/matplotlib/* # 폰트 캐시 재설정
print("=" * 50)
print("✅ 폰트 설치 완료!")
print("⚠️ 이제 다음을 수행하세요:")
print("1. 상단 메뉴: 런타임 → 런타임 다시 시작")
print("2. 재시작 후 Step 2 코드 실행")
print("=" * 50)

Step 2: 실행

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
#
path = '/usr/share/fonts/truetype/nanum/NanumGothic.ttf'
font_name = mpl.font_manager.FontProperties(fname=path).get_name()
plt.rcParams['font.family'] = font_name

# 확인
plt.plot([1,2,3], [1,2,3])
plt.title('한글 테스트 성공! 🎉')
plt.xlabel('한글 X축')
plt.ylabel('한글 Y축')
plt.show()

In [ ]:
# YOLOv8 및 필요 라이브러리 설치
!pip install ultralytics -q
!pip install opencv-python-headless pillow -q
!pip install torch torchvision -q
!pip install scikit-image scikit-learn -q
!pip install albumentations -q
!pip install labelme -q  # 라벨링 도구
!pip install roboflow -q  # 데이터셋 관리

In [ ]:
import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pathlib import Path
import json
import os
from datetime import datetime

# Deep Learning
import torch
import torchvision
from torchvision import transforms

# YOLOv8
from ultralytics import YOLO

# Image Processing
from skimage import measure, morphology, filters
from sklearn.metrics import confusion_matrix, classification_report
import albumentations as A

# 시각화 설정
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10

# GPU 확인
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"✅ Device: {device}")
if device.type == 'cuda':
    print(f"   GPU: {torch.cuda.get_device_name(0)}")

## 2. 제조 불량 데이터 시뮬레이션 및 수집

### 2.1 불량 유형 정의 및 샘플 생성

In [ ]:
class ManufacturingDefectSimulator:
    def __init__(self):
        self.defect_types = {
            'scratch': {'color': (255, 0, 0), 'severity': ['minor', 'major', 'critical']},
            'dent': {'color': (0, 255, 0), 'severity': ['shallow', 'deep']},
            'contamination': {'color': (0, 0, 255), 'severity': ['light', 'heavy']},
            'crack': {'color': (255, 255, 0), 'severity': ['hairline', 'visible', 'severe']},
            'discoloration': {'color': (255, 0, 255), 'severity': ['slight', 'moderate', 'severe']}
        }

    def generate_synthetic_image(self, width=640, height=480, defect_type=None):
        """합성 불량 이미지 생성"""
        # 기본 배경 생성 (제품 표면 시뮬레이션)
        image = np.ones((height, width, 3), dtype=np.uint8) * 200

        # 텍스처 추가
        noise = np.random.normal(0, 10, (height, width, 3))
        image = np.clip(image + noise, 0, 255).astype(np.uint8)

        annotations = []

        if defect_type:
            # 불량 추가
            if defect_type == 'scratch':
                # 스크래치 생성
                x1, y1 = np.random.randint(50, width-50), np.random.randint(50, height-50)
                x2, y2 = x1 + np.random.randint(50, 200), y1 + np.random.randint(-30, 30)
                cv2.line(image, (x1, y1), (x2, y2), self.defect_types[defect_type]['color'], 2)

                bbox = [min(x1, x2), min(y1, y2), abs(x2-x1), abs(y2-y1)+4]
                annotations.append({
                    'type': defect_type,
                    'bbox': bbox,
                    'severity': np.random.choice(self.defect_types[defect_type]['severity'])
                })

            elif defect_type == 'dent':
                # 덴트 생성
                center = (np.random.randint(50, width-50), np.random.randint(50, height-50))
                radius = np.random.randint(20, 50)
                cv2.circle(image, center, radius, self.defect_types[defect_type]['color'], -1)

                bbox = [center[0]-radius, center[1]-radius, radius*2, radius*2]
                annotations.append({
                    'type': defect_type,
                    'bbox': bbox,
                    'severity': np.random.choice(self.defect_types[defect_type]['severity'])
                })

            elif defect_type == 'contamination':
                # 오염 생성
                x, y = np.random.randint(50, width-100), np.random.randint(50, height-100)
                w, h = np.random.randint(30, 100), np.random.randint(30, 100)

                contamination = np.random.randint(100, 150, (h, w, 3))
                image[y:y+h, x:x+w] = contamination

                bbox = [x, y, w, h]
                annotations.append({
                    'type': defect_type,
                    'bbox': bbox,
                    'severity': np.random.choice(self.defect_types[defect_type]['severity'])
                })

        return image, annotations

    def generate_dataset(self, n_images=100, defect_ratio=0.3):
        """데이터셋 생성"""
        images = []
        labels = []

        for i in range(n_images):
            if np.random.random() < defect_ratio:
                # 불량 이미지
                defect_type = np.random.choice(list(self.defect_types.keys()))
                img, annot = self.generate_synthetic_image(defect_type=defect_type)
                images.append(img)
                labels.append(annot)
            else:
                # 정상 이미지
                img, _ = self.generate_synthetic_image(defect_type=None)
                images.append(img)
                labels.append([])

        return images, labels

# 불량 데이터 생성
simulator = ManufacturingDefectSimulator()
sample_images, sample_labels = simulator.generate_dataset(n_images=50, defect_ratio=0.5)

# 샘플 시각화
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Synthetic Manufacturing Defect Samples', fontsize=16, fontweight='bold')

for idx, ax in enumerate(axes.flat):
    if idx < len(sample_images):
        img = sample_images[idx]
        label = sample_labels[idx]

        ax.imshow(img)

        # 바운딩 박스 그리기
        if label:
            for annot in label:
                x, y, w, h = annot['bbox']
                rect = plt.Rectangle((x, y), w, h, linewidth=2,
                                    edgecolor='red', facecolor='none')
                ax.add_patch(rect)
                ax.text(x, y-5, f"{annot['type']} ({annot['severity']})",
                       color='red', fontsize=8, weight='bold')

        status = "Defective" if label else "Normal"
        ax.set_title(f"Sample {idx+1}: {status}")
        ax.axis('off')

plt.tight_layout()
plt.show()

print(f"✅ 생성된 데이터셋: {len(sample_images)} 이미지")
print(f"   - 불량: {sum(1 for l in sample_labels if l)} 개")
print(f"   - 정상: {sum(1 for l in sample_labels if not l)} 개")

### 2.2 조명 및 카메라 설정 가이드

In [ ]:
class VisionSystemGuide:
    def __init__(self):
        self.lighting_types = {
            'Front Light': {
                'use_case': '일반적인 표면 검사',
                'advantages': '균일한 조명, 색상 재현성 좋음',
                'disadvantages': '그림자 발생 가능'
            },
            'Back Light': {
                'use_case': '외형 검사, 구멍/크랙 검출',
                'advantages': '높은 대비, 외형 명확',
                'disadvantages': '표면 세부사항 손실'
            },
            'Coaxial Light': {
                'use_case': '반사 표면 검사',
                'advantages': '균일한 조명, 반사 최소화',
                'disadvantages': '비용 높음'
            },
            'Ring Light': {
                'use_case': '작은 부품 검사',
                'advantages': '그림자 없음, 균일한 조명',
                'disadvantages': '제한된 작업 거리'
            },
            'Dome Light': {
                'use_case': '곡면/반사 표면',
                'advantages': '매우 균일한 조명',
                'disadvantages': '크기 제한, 비용'
            }
        }

        self.camera_specs = {
            'resolution': {'min': '1920x1080', 'recommended': '2448x2048', 'high': '4096x3072'},
            'fps': {'inspection': 30, 'high_speed': 120, 'ultra_high': 500},
            'sensor_type': ['CCD', 'CMOS'],
            'pixel_size': {'standard': '3.45μm', 'high_sensitivity': '5.5μm'}
        }

    def simulate_lighting_effect(self, image, lighting_type):
        """조명 효과 시뮬레이션"""
        if lighting_type == 'Front Light':
            # 전면 조명 효과
            result = cv2.addWeighted(image, 1.2, np.ones_like(image)*255, 0.1, 0)

        elif lighting_type == 'Back Light':
            # 후면 조명 효과 (실루엣)
            gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
            _, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
            result = cv2.cvtColor(binary, cv2.COLOR_GRAY2RGB)

        elif lighting_type == 'Coaxial Light':
            # 동축 조명 효과
            result = cv2.bilateralFilter(image, 9, 75, 75)

        elif lighting_type == 'Ring Light':
            # 링 조명 효과
            kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
            result = cv2.morphologyEx(image, cv2.MORPH_GRADIENT, kernel)
            result = cv2.addWeighted(image, 0.7, result, 0.3, 0)

        else:  # Dome Light
            # 돔 조명 효과
            result = cv2.GaussianBlur(image, (3, 3), 0)
            result = cv2.addWeighted(result, 1.1, np.ones_like(image)*255, 0.05, 0)

        return np.clip(result, 0, 255).astype(np.uint8)

    def recommend_setup(self, defect_type):
        """불량 유형별 최적 세팅 추천"""
        recommendations = {
            'scratch': {
                'lighting': 'Coaxial Light or Low-angle Light',
                'camera': '고해상도 (>5MP), 매크로 렌즈',
                'processing': 'Edge detection, Gradient analysis'
            },
            'dent': {
                'lighting': '3D structured light or Shadow technique',
                'camera': 'Stereo vision or 3D camera',
                'processing': 'Depth mapping, Surface reconstruction'
            },
            'contamination': {
                'lighting': 'UV light or Polarized light',
                'camera': 'Color camera with good SNR',
                'processing': 'Color segmentation, Texture analysis'
            },
            'crack': {
                'lighting': 'Back light or Transmitted light',
                'camera': 'High resolution, Telecentric lens',
                'processing': 'Morphological operations, Line detection'
            },
            'discoloration': {
                'lighting': 'Diffuse dome light',
                'camera': 'Calibrated color camera',
                'processing': 'Color space analysis, Statistical methods'
            }
        }

        return recommendations.get(defect_type, {})

# 조명 효과 시뮬레이션
vision_guide = VisionSystemGuide()

# 원본 이미지에 다양한 조명 효과 적용
test_image = sample_images[0]
lighting_effects = ['Front Light', 'Back Light', 'Coaxial Light', 'Ring Light', 'Dome Light']

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Different Lighting Effects on Defect Detection', fontsize=16, fontweight='bold')

axes[0, 0].imshow(test_image)
axes[0, 0].set_title('Original Image')
axes[0, 0].axis('off')

for idx, (ax, light_type) in enumerate(zip(axes.flat[1:], lighting_effects)):
    processed = vision_guide.simulate_lighting_effect(test_image, light_type)
    ax.imshow(processed)
    ax.set_title(f'{light_type}')
    ax.axis('off')

plt.tight_layout()
plt.show()

# 불량 유형별 추천 설정 출력
print("\n📸 불량 유형별 최적 비전 시스템 설정:\n")
for defect in ['scratch', 'dent', 'contamination']:
    rec = vision_guide.recommend_setup(defect)
    print(f"🔍 {defect.upper()}:")
    for key, value in rec.items():
        print(f"   - {key}: {value}")
    print()

### 2.3 데이터 라벨링 품질 관리

In [ ]:
class LabelQualityManager:
    def __init__(self):
        self.labeling_guidelines = {
            'consistency': 'Same defect type should be labeled identically across images',
            'completeness': 'All visible defects must be labeled',
            'accuracy': 'Bounding boxes should tightly fit defects',
            'granularity': 'Use appropriate class hierarchy (defect → type → severity)'
        }

        self.quality_metrics = {}

    def check_label_quality(self, annotations):
        """라벨 품질 검사"""
        issues = []

        for idx, annot in enumerate(annotations):
            if not annot:
                continue

            for item in annot:
                bbox = item.get('bbox', [])

                # 바운딩 박스 크기 검사
                if len(bbox) == 4:
                    x, y, w, h = bbox

                    # 너무 작은 박스
                    if w < 10 or h < 10:
                        issues.append({
                            'image_idx': idx,
                            'issue': 'Bounding box too small',
                            'bbox': bbox
                        })

                    # 비정상적 비율
                    aspect_ratio = w / h if h > 0 else 0
                    if aspect_ratio > 10 or aspect_ratio < 0.1:
                        issues.append({
                            'image_idx': idx,
                            'issue': 'Abnormal aspect ratio',
                            'bbox': bbox,
                            'ratio': aspect_ratio
                        })

                # 라벨 누락 검사
                if not item.get('type'):
                    issues.append({
                        'image_idx': idx,
                        'issue': 'Missing defect type'
                    })

        return issues

    def calculate_inter_annotator_agreement(self, annotations1, annotations2):
        """라벨러 간 일치도 계산 (IoU 기반)"""
        agreements = []

        for ann1, ann2 in zip(annotations1, annotations2):
            if not ann1 and not ann2:
                agreements.append(1.0)  # 둘 다 정상으로 판정
            elif not ann1 or not ann2:
                agreements.append(0.0)  # 불일치
            else:
                # IoU 계산
                iou_scores = []
                for box1 in ann1:
                    for box2 in ann2:
                        iou = self.calculate_iou(box1['bbox'], box2['bbox'])
                        if box1['type'] == box2['type']:
                            iou_scores.append(iou)
                        else:
                            iou_scores.append(0)

                if iou_scores:
                    agreements.append(np.mean(iou_scores))
                else:
                    agreements.append(0.0)

        return np.mean(agreements)

    def calculate_iou(self, box1, box2):
        """IoU (Intersection over Union) 계산"""
        x1, y1, w1, h1 = box1
        x2, y2, w2, h2 = box2

        # 교집합 영역
        xi1 = max(x1, x2)
        yi1 = max(y1, y2)
        xi2 = min(x1 + w1, x2 + w2)
        yi2 = min(y1 + h1, y2 + h2)

        if xi2 < xi1 or yi2 < yi1:
            return 0.0

        intersection = (xi2 - xi1) * (yi2 - yi1)
        union = w1 * h1 + w2 * h2 - intersection

        return intersection / union if union > 0 else 0

    def generate_labeling_report(self, annotations):
        """라벨링 품질 리포트 생성"""
        total_images = len(annotations)
        labeled_images = sum(1 for ann in annotations if ann)
        total_defects = sum(len(ann) for ann in annotations if ann)

        defect_distribution = {}
        for ann in annotations:
            if ann:
                for item in ann:
                    defect_type = item.get('type', 'unknown')
                    defect_distribution[defect_type] = defect_distribution.get(defect_type, 0) + 1

        report = {
            'total_images': total_images,
            'labeled_images': labeled_images,
            'labeling_rate': labeled_images / total_images * 100,
            'total_defects': total_defects,
            'avg_defects_per_image': total_defects / labeled_images if labeled_images > 0 else 0,
            'defect_distribution': defect_distribution
        }

        return report

# 라벨 품질 검사
label_manager = LabelQualityManager()

# 품질 이슈 검사
quality_issues = label_manager.check_label_quality(sample_labels)
print("🔍 라벨 품질 검사 결과:")
print(f"   발견된 이슈: {len(quality_issues)} 개\n")

if quality_issues[:3]:
    print("   샘플 이슈:")
    for issue in quality_issues[:3]:
        print(f"   - Image {issue['image_idx']}: {issue['issue']}")

# 라벨링 리포트
report = label_manager.generate_labeling_report(sample_labels)
print("\n📊 라벨링 통계 리포트:")
for key, value in report.items():
    if key != 'defect_distribution':
        print(f"   {key}: {value:.2f}" if isinstance(value, float) else f"   {key}: {value}")

print("\n   불량 유형 분포:")
for defect_type, count in report['defect_distribution'].items():
    print(f"     - {defect_type}: {count} 개")

## 3. YOLO 모델 학습 및 전이학습

### 3.1 YOLO 모델 준비 및 데이터 포맷 변환

In [ ]:
class YOLODataPreparation:
    def __init__(self, images, labels):
        self.images = images
        self.labels = labels
        self.class_names = ['scratch', 'dent', 'contamination', 'crack', 'discoloration']
        self.class_to_id = {name: idx for idx, name in enumerate(self.class_names)}

    def convert_to_yolo_format(self, bbox, img_width, img_height, class_name):
        """바운딩 박스를 YOLO 포맷으로 변환"""
        x, y, w, h = bbox

        # 중심점 좌표와 크기를 정규화
        center_x = (x + w/2) / img_width
        center_y = (y + h/2) / img_height
        norm_w = w / img_width
        norm_h = h / img_height

        class_id = self.class_to_id.get(class_name, 0)

        return f"{class_id} {center_x:.6f} {center_y:.6f} {norm_w:.6f} {norm_h:.6f}"

    def prepare_dataset(self, split_ratio=0.8):
        """학습/검증 데이터셋 준비"""
        n_samples = len(self.images)
        n_train = int(n_samples * split_ratio)

        # 데이터 셔플
        indices = np.random.permutation(n_samples)

        train_indices = indices[:n_train]
        val_indices = indices[n_train:]

        # 디렉토리 생성
        os.makedirs('dataset/train/images', exist_ok=True)
        os.makedirs('dataset/train/labels', exist_ok=True)
        os.makedirs('dataset/val/images', exist_ok=True)
        os.makedirs('dataset/val/labels', exist_ok=True)

        # 학습 데이터 저장
        for idx in train_indices:
            img = self.images[idx]
            label = self.labels[idx]

            # 이미지 저장
            img_path = f'dataset/train/images/img_{idx:04d}.jpg'
            cv2.imwrite(img_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

            # 라벨 저장 (YOLO 포맷)
            label_path = f'dataset/train/labels/img_{idx:04d}.txt'
            with open(label_path, 'w') as f:
                if label:
                    h, w = img.shape[:2]
                    for annot in label:
                        yolo_line = self.convert_to_yolo_format(
                            annot['bbox'], w, h, annot['type']
                        )
                        f.write(yolo_line + '\n')

        # 검증 데이터 저장
        for idx in val_indices:
            img = self.images[idx]
            label = self.labels[idx]

            # 이미지 저장
            img_path = f'dataset/val/images/img_{idx:04d}.jpg'
            cv2.imwrite(img_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

            # 라벨 저장
            label_path = f'dataset/val/labels/img_{idx:04d}.txt'
            with open(label_path, 'w') as f:
                if label:
                    h, w = img.shape[:2]
                    for annot in label:
                        yolo_line = self.convert_to_yolo_format(
                            annot['bbox'], w, h, annot['type']
                        )
                        f.write(yolo_line + '\n')

        # YAML 설정 파일 생성
        yaml_content = f"""# Manufacturing Defect Detection Dataset
path: dataset
train: train/images
val: val/images

# Classes
nc: {len(self.class_names)}
names: {self.class_names}
"""

        with open('dataset/defect_data.yaml', 'w') as f:
            f.write(yaml_content)

        print(f"✅ 데이터셋 준비 완료")
        print(f"   - Training: {len(train_indices)} images")
        print(f"   - Validation: {len(val_indices)} images")

        return train_indices, val_indices

# YOLO 데이터셋 준비
yolo_prep = YOLODataPreparation(sample_images, sample_labels)
train_idx, val_idx = yolo_prep.prepare_dataset(split_ratio=0.8)

### 3.2 YOLO 모델 학습 및 전이학습

In [ ]:
# YOLOv8 모델 초기화 및 전이학습
class YOLODefectDetector:
    def __init__(self, model_size='n'):
        """
        model_size: 'n'(nano), 's'(small), 'm'(medium), 'l'(large), 'x'(extra large)
        """
        self.model_size = model_size
        self.model = None
        self.results = None

    def initialize_pretrained_model(self):
        """사전 학습된 YOLO 모델 로드"""
        model_name = f'yolov8{self.model_size}.pt'
        self.model = YOLO(model_name)
        print(f"✅ Loaded pretrained model: {model_name}")

    def train_model(self, data_yaml, epochs=10, imgsz=640, batch=8):
        """모델 학습"""
        if self.model is None:
            self.initialize_pretrained_model()

        # 학습 파라미터
        self.results = self.model.train(
            data=data_yaml,
            epochs=epochs,
            imgsz=imgsz,
            batch=batch,
            patience=5,
            save=True,
            device=device.type,
            verbose=True,
            # 추가 하이퍼파라미터
            lr0=0.01,  # 초기 학습률
            lrf=0.01,  # 최종 학습률 (lr0 * lrf)
            momentum=0.937,
            weight_decay=0.0005,
            warmup_epochs=3,
            warmup_momentum=0.8,
            box=7.5,  # box loss gain
            cls=0.5,  # cls loss gain
            degrees=0.0,  # 회전 증강
            translate=0.1,  # 이동 증강
            scale=0.5,  # 스케일 증강
            flipud=0.0,  # 상하 반전
            fliplr=0.5,  # 좌우 반전
            mosaic=1.0,  # 모자이크 증강
            mixup=0.0  # MixUp 증강
        )

        print("\n✅ 모델 학습 완료!")
        return self.results

    def evaluate_model(self):
        """모델 평가"""
        if self.model is None:
            print("❌ 모델이 학습되지 않았습니다.")
            return

        # 검증 데이터로 평가
        metrics = self.model.val()

        print("\n📊 모델 평가 결과:")
        print(f"   - mAP50: {metrics.box.map50:.4f}")
        print(f"   - mAP50-95: {metrics.box.map:.4f}")

        return metrics

    def predict(self, image, conf_threshold=0.25):
        """단일 이미지 예측"""
        if self.model is None:
            print("❌ 모델이 로드되지 않았습니다.")
            return None

        results = self.model(image, conf=conf_threshold)
        return results[0]

    def visualize_predictions(self, image, predictions):
        """예측 결과 시각화"""
        img_copy = image.copy()

        if predictions.boxes is not None:
            boxes = predictions.boxes.xyxy.cpu().numpy()
            confidences = predictions.boxes.conf.cpu().numpy()
            classes = predictions.boxes.cls.cpu().numpy()

            for box, conf, cls in zip(boxes, confidences, classes):
                x1, y1, x2, y2 = map(int, box)
                class_name = yolo_prep.class_names[int(cls)]

                # 바운딩 박스 그리기
                cv2.rectangle(img_copy, (x1, y1), (x2, y2), (0, 255, 0), 2)

                # 라벨 추가
                label = f"{class_name}: {conf:.2f}"
                cv2.putText(img_copy, label, (x1, y1-10),
                           cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

        return img_copy

# YOLO 모델 학습 (시뮬레이션 - 실제로는 더 많은 에폭이 필요)
detector = YOLODefectDetector(model_size='n')  # nano 모델 사용 (빠른 학습)
detector.initialize_pretrained_model()

# 참고: 실제 학습은 많은 시간이 걸리므로 여기서는 시뮬레이션만 수행
print("\n📝 학습 설정 예시:")
print("detector.train_model('dataset/defect_data.yaml', epochs=50, batch=16)")
print("\n💡 실제 프로덕션에서는 다음을 권장:")
print("   - epochs: 100-300")
print("   - batch size: GPU 메모리에 따라 조정 (16-64)")
print("   - image size: 640x640 또는 1280x1280")
print("   - model size: 's' 또는 'm' (정확도와 속도의 균형)")

## 4. 세그멘테이션 및 고급 검사 기법

### 4.1 Instance Segmentation for Precise Defect Analysis

In [ ]:
class DefectSegmentation:
    def __init__(self):
        self.segmentation_methods = {
            'threshold': 'Simple threshold-based segmentation',
            'watershed': 'Watershed algorithm for touching objects',
            'grabcut': 'GrabCut for foreground extraction',
            'deep_learning': 'U-Net, Mask R-CNN for precise segmentation'
        }

    def threshold_segmentation(self, image, lower_thresh=100, upper_thresh=200):
        """임계값 기반 세그멘테이션"""
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        _, binary = cv2.threshold(gray, lower_thresh, upper_thresh, cv2.THRESH_BINARY)

        # 노이즈 제거
        kernel = np.ones((3, 3), np.uint8)
        binary = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel)
        binary = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)

        return binary

    def watershed_segmentation(self, image):
        """Watershed 알고리즘"""
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

        # 노이즈 제거
        denoised = cv2.medianBlur(gray, 5)

        # Threshold
        _, thresh = cv2.threshold(denoised, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

        # 확실한 배경 영역 찾기
        kernel = np.ones((3, 3), np.uint8)
        sure_bg = cv2.dilate(thresh, kernel, iterations=3)

        # 확실한 전경 영역 찾기
        dist_transform = cv2.distanceTransform(thresh, cv2.DIST_L2, 5)
        _, sure_fg = cv2.threshold(dist_transform, 0.3*dist_transform.max(), 255, 0)
        sure_fg = np.uint8(sure_fg)

        # 불확실한 영역 찾기
        unknown = cv2.subtract(sure_bg, sure_fg)

        # 마커 생성
        _, markers = cv2.connectedComponents(sure_fg)
        markers = markers + 1
        markers[unknown == 255] = 0

        # Watershed 적용
        markers = cv2.watershed(image, markers)

        # 경계선 표시
        result = image.copy()
        result[markers == -1] = [255, 0, 0]

        return markers, result

    def extract_defect_features(self, image, mask):
        """불량 영역의 특징 추출"""
        # 연결된 구성요소 찾기
        labeled = measure.label(mask)
        regions = measure.regionprops(labeled, intensity_image=cv2.cvtColor(image, cv2.COLOR_RGB2GRAY))

        features = []
        for region in regions:
            feature = {
                'area': region.area,
                'perimeter': region.perimeter,
                'centroid': region.centroid,
                'eccentricity': region.eccentricity,
                'solidity': region.solidity,
                'mean_intensity': region.mean_intensity,
                'bbox': region.bbox,
                'major_axis_length': region.major_axis_length,
                'minor_axis_length': region.minor_axis_length
            }

            # 형태 분류
            if feature['eccentricity'] > 0.9:
                feature['shape_type'] = 'linear'  # 스크래치 유형
            elif feature['solidity'] < 0.5:
                feature['shape_type'] = 'irregular'  # 불규칙한 오염
            else:
                feature['shape_type'] = 'blob'  # 덴트, 얼룩 등

            features.append(feature)

        return features

# 세그멘테이션 적용
segmentor = DefectSegmentation()

# 테스트 이미지에 다양한 세그멘테이션 적용
test_img = sample_images[1]

# Threshold segmentation
thresh_result = segmentor.threshold_segmentation(test_img)

# Watershed segmentation
watershed_markers, watershed_result = segmentor.watershed_segmentation(test_img)

# 특징 추출
features = segmentor.extract_defect_features(test_img, thresh_result)

# 시각화
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Segmentation Methods for Defect Detection', fontsize=16, fontweight='bold')

axes[0, 0].imshow(test_img)
axes[0, 0].set_title('Original Image')
axes[0, 0].axis('off')

axes[0, 1].imshow(thresh_result, cmap='gray')
axes[0, 1].set_title('Threshold Segmentation')
axes[0, 1].axis('off')

axes[0, 2].imshow(watershed_result)
axes[0, 2].set_title('Watershed Segmentation')
axes[0, 2].axis('off')

# Edge detection
edges = cv2.Canny(cv2.cvtColor(test_img, cv2.COLOR_RGB2GRAY), 50, 150)
axes[1, 0].imshow(edges, cmap='gray')
axes[1, 0].set_title('Edge Detection (Canny)')
axes[1, 0].axis('off')

# Morphological operations
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
morph = cv2.morphologyEx(thresh_result, cv2.MORPH_GRADIENT, kernel)
axes[1, 1].imshow(morph, cmap='gray')
axes[1, 1].set_title('Morphological Gradient')
axes[1, 1].axis('off')

# Feature visualization
feature_img = test_img.copy()
for feat in features[:5]:  # 상위 5개 특징만 표시
    if 'centroid' in feat:
        cy, cx = feat['centroid']
        cv2.circle(feature_img, (int(cx), int(cy)), 5, (255, 0, 0), -1)

axes[1, 2].imshow(feature_img)
axes[1, 2].set_title('Detected Features')
axes[1, 2].axis('off')

plt.tight_layout()
plt.show()

print("📈 추출된 불량 특징 (상위 3개):")
for i, feat in enumerate(features[:3]):
    print(f"\n불량 {i+1}:")
    print(f"  - 면적: {feat.get('area', 0):.0f} pixels")
    print(f"  - 둘레: {feat.get('perimeter', 0):.2f}")
    print(f"  - 형태: {feat.get('shape_type', 'unknown')}")
    print(f"  - 편심률: {feat.get('eccentricity', 0):.3f}")

## 5. 오탐/미탐 트레이드오프 최적화

### 5.1 임계값 튜닝 및 성능 평가

In [ ]:
class ThresholdOptimizer:
    def __init__(self):
        self.metrics_history = []

    def calculate_metrics(self, y_true, y_pred, threshold=0.5):
        """성능 지표 계산"""
        # 이진 분류로 변환
        y_pred_binary = (y_pred >= threshold).astype(int)

        # Confusion Matrix
        tn, fp, fn, tp = confusion_matrix(y_true, y_pred_binary).ravel()

        # 주요 지표 계산
        metrics = {
            'threshold': threshold,
            'true_positive': tp,
            'false_positive': fp,  # 오탐 (Type I Error)
            'true_negative': tn,
            'false_negative': fn,  # 미탐 (Type II Error)
            'accuracy': (tp + tn) / (tp + tn + fp + fn) if (tp + tn + fp + fn) > 0 else 0,
            'precision': tp / (tp + fp) if (tp + fp) > 0 else 0,
            'recall': tp / (tp + fn) if (tp + fn) > 0 else 0,  # 검출률
            'f1_score': 0,
            'false_positive_rate': fp / (fp + tn) if (fp + tn) > 0 else 0,  # 오탐률
            'false_negative_rate': fn / (fn + tp) if (fn + tp) > 0 else 0   # 미탐률
        }

        # F1 Score
        if metrics['precision'] + metrics['recall'] > 0:
            metrics['f1_score'] = 2 * (metrics['precision'] * metrics['recall']) / \
                                  (metrics['precision'] + metrics['recall'])

        return metrics

    def optimize_threshold(self, y_true, y_scores, threshold_range=None, optimization_target='f1'):
        """최적 임계값 찾기"""
        if threshold_range is None:
            threshold_range = np.arange(0.1, 0.95, 0.05)

        best_threshold = 0.5
        best_score = 0

        for threshold in threshold_range:
            metrics = self.calculate_metrics(y_true, y_scores, threshold)
            self.metrics_history.append(metrics)

            # 최적화 대상에 따른 점수 계산
            if optimization_target == 'f1':
                score = metrics['f1_score']
            elif optimization_target == 'precision':
                score = metrics['precision']
            elif optimization_target == 'recall':
                score = metrics['recall']
            elif optimization_target == 'balanced':
                # 오탐과 미탐의 균형
                score = 1 - (metrics['false_positive_rate'] + metrics['false_negative_rate']) / 2
            else:
                score = metrics['accuracy']

            if score > best_score:
                best_score = score
                best_threshold = threshold

        return best_threshold, best_score

    def plot_tradeoff_curve(self):
        """오탐/미탐 트레이드오프 커브 시각화"""
        if not self.metrics_history:
            print("No metrics history available")
            return

        df = pd.DataFrame(self.metrics_history)

        fig, axes = plt.subplots(2, 2, figsize=(15, 12))
        fig.suptitle('Threshold Optimization Analysis', fontsize=16, fontweight='bold')

        # 1. Precision vs Recall
        axes[0, 0].plot(df['threshold'], df['precision'], 'b-', label='Precision', linewidth=2)
        axes[0, 0].plot(df['threshold'], df['recall'], 'r-', label='Recall', linewidth=2)
        axes[0, 0].plot(df['threshold'], df['f1_score'], 'g--', label='F1 Score', linewidth=2)
        axes[0, 0].set_xlabel('Threshold')
        axes[0, 0].set_ylabel('Score')
        axes[0, 0].set_title('Precision, Recall, and F1 Score vs Threshold')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)

        # 2. False Positive vs False Negative
        axes[0, 1].plot(df['threshold'], df['false_positive_rate'], 'orange',
                       label='False Positive Rate (오탐률)', linewidth=2)
        axes[0, 1].plot(df['threshold'], df['false_negative_rate'], 'purple',
                       label='False Negative Rate (미탐률)', linewidth=2)
        axes[0, 1].set_xlabel('Threshold')
        axes[0, 1].set_ylabel('Error Rate')
        axes[0, 1].set_title('오탐률 vs 미탐률')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)

        # 3. ROC-like curve (FPR vs TPR)
        tpr = df['recall']  # True Positive Rate = Recall
        fpr = df['false_positive_rate']
        axes[1, 0].plot(fpr, tpr, 'b-', linewidth=2)
        axes[1, 0].plot([0, 1], [0, 1], 'r--', alpha=0.5)
        axes[1, 0].set_xlabel('False Positive Rate')
        axes[1, 0].set_ylabel('True Positive Rate')
        axes[1, 0].set_title('ROC-like Curve')
        axes[1, 0].grid(True, alpha=0.3)

        # 4. Cost Analysis
        # 비용 가정: 오탐 비용 = 1, 미탐 비용 = 10 (미탐이 더 비싸다고 가정)
        cost_fp = 1  # 오탐 비용
        cost_fn = 10  # 미탐 비용

        total_cost = df['false_positive'] * cost_fp + df['false_negative'] * cost_fn
        axes[1, 1].plot(df['threshold'], total_cost, 'r-', linewidth=2)
        axes[1, 1].set_xlabel('Threshold')
        axes[1, 1].set_ylabel('Total Cost')
        axes[1, 1].set_title(f'Cost Analysis (FP cost={cost_fp}, FN cost={cost_fn})')
        axes[1, 1].grid(True, alpha=0.3)

        # 최적점 표시
        optimal_idx = df['f1_score'].idxmax()
        optimal_threshold = df.loc[optimal_idx, 'threshold']

        for ax in axes.flat:
            ax.axvline(x=optimal_threshold, color='green', linestyle=':',
                      alpha=0.7, label=f'Optimal ({optimal_threshold:.2f})')

        plt.tight_layout()
        plt.show()

        return df

# 임계값 최적화 시뮬레이션
np.random.seed(42)

# 시뮬레이션 데이터 생성
n_samples = 1000
y_true = np.random.randint(0, 2, n_samples)  # 실제 라벨 (0: 정상, 1: 불량)
y_scores = np.random.beta(2, 5, n_samples)  # 모델의 신뢰도 점수
y_scores[y_true == 1] += np.random.normal(0.3, 0.1, sum(y_true == 1))  # 불량품은 높은 점수
y_scores = np.clip(y_scores, 0, 1)

# 최적화 실행
optimizer = ThresholdOptimizer()

# 다양한 최적화 목표로 테스트
optimization_targets = ['f1', 'precision', 'recall', 'balanced']

print("🎯 임계값 최적화 결과:\n")
for target in optimization_targets:
    best_threshold, best_score = optimizer.optimize_threshold(
        y_true, y_scores, optimization_target=target
    )
    print(f"목표: {target:10s} | 최적 임계값: {best_threshold:.3f} | 점수: {best_score:.3f}")

# 트레이드오프 곡선 시각화
metrics_df = optimizer.plot_tradeoff_curve()

### 5.2 실전 최적화 전략

In [ ]:
class ProductionOptimizationStrategy:
    def __init__(self):
        self.strategies = {
            'conservative': {
                'description': '미탐 최소화 (고품질 요구)',
                'threshold': 0.3,
                'use_case': '의료기기, 자동차 안전부품'
            },
            'balanced': {
                'description': '균형잡힌 접근',
                'threshold': 0.5,
                'use_case': '일반 제조업'
            },
            'aggressive': {
                'description': '오탐 최소화 (비용 절감)',
                'threshold': 0.7,
                'use_case': '저가 대량생산 제품'
            },
            'dynamic': {
                'description': '실시간 조정',
                'threshold': 'adaptive',
                'use_case': '변동성 높은 생산 라인'
            }
        }

    def recommend_strategy(self, product_info):
        """제품 특성에 따른 전략 추천"""
        defect_cost = product_info.get('defect_cost', 'medium')
        production_volume = product_info.get('volume', 'medium')
        quality_requirement = product_info.get('quality', 'medium')

        if quality_requirement == 'critical' or defect_cost == 'very_high':
            return 'conservative'
        elif production_volume == 'very_high' and defect_cost == 'low':
            return 'aggressive'
        elif production_volume == 'variable':
            return 'dynamic'
        else:
            return 'balanced'

    def calculate_business_impact(self, metrics, production_info):
        """비즈니스 영향도 계산"""
        daily_production = production_info.get('daily_volume', 10000)
        unit_price = production_info.get('unit_price', 100)
        defect_cost_multiplier = production_info.get('defect_multiplier', 10)
        inspection_cost = production_info.get('inspection_cost', 1)

        # 일일 비용 계산
        false_positives = metrics['false_positive_rate'] * daily_production
        false_negatives = metrics['false_negative_rate'] * daily_production

        # 오탐 비용: 재검사 비용 + 생산 지연
        fp_cost = false_positives * inspection_cost * 2

        # 미탐 비용: 불량품 출하로 인한 손실
        fn_cost = false_negatives * unit_price * defect_cost_multiplier

        total_cost = fp_cost + fn_cost

        return {
            'daily_false_positives': int(false_positives),
            'daily_false_negatives': int(false_negatives),
            'false_positive_cost': fp_cost,
            'false_negative_cost': fn_cost,
            'total_daily_cost': total_cost,
            'annual_cost': total_cost * 250  # 연간 작업일 250일 가정
        }

# 실전 최적화 전략 시뮬레이션
strategy_optimizer = ProductionOptimizationStrategy()

# 다양한 제품 시나리오
product_scenarios = [
    {
        'name': '의료용 임플란트',
        'defect_cost': 'very_high',
        'volume': 'low',
        'quality': 'critical',
        'daily_volume': 100,
        'unit_price': 10000,
        'defect_multiplier': 100,
        'inspection_cost': 50
    },
    {
        'name': '스마트폰 케이스',
        'defect_cost': 'low',
        'volume': 'very_high',
        'quality': 'medium',
        'daily_volume': 50000,
        'unit_price': 10,
        'defect_multiplier': 3,
        'inspection_cost': 0.1
    },
    {
        'name': '자동차 브레이크 패드',
        'defect_cost': 'high',
        'volume': 'medium',
        'quality': 'high',
        'daily_volume': 5000,
        'unit_price': 500,
        'defect_multiplier': 50,
        'inspection_cost': 5
    }
]

print("🏭 제품별 최적화 전략 추천:\n")
for product in product_scenarios:
    recommended = strategy_optimizer.recommend_strategy(product)
    strategy = strategy_optimizer.strategies[recommended]

    print(f"📦 {product['name']}:")
    print(f"   추천 전략: {recommended} - {strategy['description']}")
    print(f"   권장 임계값: {strategy['threshold']}")
    print(f"   적용 사례: {strategy['use_case']}")

    # 비즈니스 영향도 계산 (예시 메트릭 사용)
    sample_metrics = {
        'false_positive_rate': 0.02 if recommended == 'aggressive' else 0.05,
        'false_negative_rate': 0.05 if recommended == 'conservative' else 0.01
    }

    impact = strategy_optimizer.calculate_business_impact(sample_metrics, product)

    print(f"   예상 일일 오탐: {impact['daily_false_positives']} 개")
    print(f"   예상 일일 미탐: {impact['daily_false_negatives']} 개")
    print(f"   일일 총 비용: ${impact['total_daily_cost']:,.0f}")
    print(f"   연간 비용: ${impact['annual_cost']:,.0f}\n")

## 6. 실습 과제 및 체크리스트

### Session 2 실습 과제

In [ ]:
# 실습 체크리스트 생성
def create_practice_checklist():
    checklist = {
        '데이터 수집': [
            '불량 샘플 이미지 최소 100장 수집',
            '정상 샘플 이미지 최소 300장 수집',
            '다양한 조명 조건에서 촬영',
            '일관된 해상도와 포맷 유지'
        ],
        '라벨링': [
            '라벨링 가이드라인 문서 작성',
            '최소 2명의 라벨러로 교차 검증',
            'IoU > 0.5 이상 일치도 확보',
            '각 불량 유형별 최소 20개 샘플'
        ],
        '모델 학습': [
            'YOLO 모델 전이학습 수행',
            'Train/Val/Test 분할 (6:2:2)',
            '최소 50 epoch 학습',
            'mAP > 0.7 달성'
        ],
        '최적화': [
            '제품별 비용 매트릭스 작성',
            '임계값 그리드 서치 수행',
            '오탐/미탐 트레이드오프 분석',
            'A/B 테스트 계획 수립'
        ],
        '배포 준비': [
            '추론 속도 측정 (FPS)',
            'Edge 디바이스 호환성 테스트',
            '모델 경량화 (Quantization)',
            '모니터링 대시보드 구성'
        ]
    }

    return checklist

# 체크리스트 출력
checklist = create_practice_checklist()

print("✅ 비전 불량검사 구축 체크리스트\n")
for category, items in checklist.items():
    print(f"📌 {category}:")
    for item in items:
        print(f"   □ {item}")
    print()

# 실습 과제 템플릿 저장
practice_template = {
    'project_info': {
        'product_name': '',
        'defect_types': [],
        'current_inspection_method': '',
        'target_metrics': {
            'detection_rate': 0,
            'false_positive_rate': 0,
            'inspection_time': 0
        }
    },
    'data_collection': {
        'camera_specs': {},
        'lighting_setup': '',
        'total_images': 0,
        'labeling_tool': ''
    },
    'model_performance': {
        'mAP50': 0,
        'inference_time': 0,
        'optimal_threshold': 0
    },
    'roi_analysis': {
        'implementation_cost': 0,
        'annual_savings': 0,
        'payback_period': 0
    }
}

# JSON 파일로 저장
with open('vision_inspection_template.json', 'w', encoding='utf-8') as f:
    json.dump(practice_template, f, indent=2, ensure_ascii=False)

print("\n💾 실습 템플릿이 'vision_inspection_template.json'으로 저장되었습니다.")
print("📝 귀사의 실제 데이터로 채워주세요.")

## 7. Session 2 핵심 요약

### 이번 세션에서 학습한 내용:

1. **데이터 수집 및 품질 관리**
   - 조명 설정의 중요성 (Front, Back, Coaxial, Ring, Dome)
   - 라벨링 품질 관리 (일관성, 완전성, 정확성)
   - Inter-annotator agreement 측정

2. **YOLO 모델 활용**
   - 전이학습을 통한 빠른 학습
   - 데이터 포맷 변환 및 증강
   - 모델 크기별 성능/속도 트레이드오프

3. **세그멘테이션 기법**
   - Threshold, Watershed 등 전통적 방법
   - 불량 특징 추출 (면적, 둘레, 편심률 등)
   - 형태 기반 불량 분류

4. **임계값 최적화**
   - 오탐/미탐 트레이드오프 분석
   - 제품별 최적화 전략 수립
   - 비즈니스 영향도 계산

### 실전 적용 팁:
- 초기에는 작은 데이터셋으로 시작하여 점진적 확대
- 현장 작업자와 긴밀한 협업으로 실용적 요구사항 파악
- 지속적인 모니터링과 재학습 체계 구축
- Edge 배포를 고려한 모델 경량화

### 다음 세션 예고:
**Session 3: 예지보전(시계열) 심화 (4-6시간)**
- FFT 및 통계 피처 추출
- Isolation Forest vs LSTM/AutoEncoder
- 실시간 이상 탐지 파이프라인 구축

In [ ]:
print("🎯 Session 2 완료!")
print("\n다음 세션 준비사항:")
print("1. 설비 센서 데이터 (진동, 온도, 전류 등) 준비")
print("2. 고장 이력 데이터 수집")
print("3. 정상/비정상 운전 조건 정의")
print("\n💡 실습 중 질문이 있으시면 언제든 문의해주세요!")